In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pytz
import names
import numpy as np
import datetime as dt
from redshift.client import RedShift

In [ ]:
host = 'vuk-redshift-cluster.cxneerbzcdew.eu-west-3.redshift.amazonaws.com'
port = '5439'
database_name = 'dev'
user = os.environ['AWSUSER']
password = os.environ['AWSUSER_PASSWD']

rs_client = RedShift(host, port, database_name, user, password)

# Create tables

In [ ]:
q_create_table_users = \
("CREATE TABLE users ("
    "full_name VARCHAR(255), "
    "age INT"
");")
q_create_table_users

In [ ]:
rs_client.execute_query(q_create_table_users)

In [ ]:
q_create_table_purchases = \
("CREATE TABLE purchases ("
    "purchase_time TIMESTAMP, "
    "full_name VARCHAR(255), "
    "product VARCHAR(255)"
");")
q_create_table_purchases

In [ ]:
rs_client.execute_query(q_create_table_purchases)

# Insert data

In [ ]:
def data_to_query(lst):
    
    def convert_value(v):
        if isinstance(v, str):
            return "'{}'".format(v)
        elif isinstance(v, int):
            return str(v)
        elif isinstance(v, dt.datetime):
            return "'{}'".format(v)
    
    data_string = "(" + "), (".join([", ".join(map(convert_value, row)) for row in lst]) + ")"
    return data_string

In [ ]:
users_data = [[names.get_first_name(), np.random.randint(18, 60)] for i in range(10)]

q_insert_into_users = \
("INSERT INTO users ("
    "full_name, "
    "age"
") "
"VALUES "
    "{data}"
";").format(data=data_to_query(users_data))
q_insert_into_users

In [ ]:
rs_client.execute_query(q_insert_into_users)

In [ ]:
purchases_data = [(dt.datetime(2018, 1, np.random.randint(1, 32), np.random.randint(0, 24), tzinfo=pytz.UTC),
                  names.get_first_name(),
                  'product{}'.format(np.random.randint(1,4))) for i in range(10)]

q_insert_into_purchases = \
("INSERT INTO purchases ("
    "purchase_time, "
    "full_name, "
    "product"
") "
"VALUES "
    "{data}"
";").format(data=data_to_query(purchases_data))
q_insert_into_purchases

In [ ]:
rs_client.execute_query(q_insert_into_purchases)

# Query data

In [ ]:
q_users_over_30 =  "SELECT * FROM public.users WHERE age > 30"

data = rs_client.get_data(q_users_over_30)
data

In [ ]:
q_purchases_after_20180118 = "SELECT * FROM public.purchases WHERE purchase_time > '2018-01-18 00:00:00+0000'"

data = rs_client.get_data(q_purchases_after_20180118)
data

# Close connection

In [ ]:
rs_client.close_connection()